In [1]:
import pickle
import pandas as pd
import time
from datasketch import MinHash, MinHashLSHForest
import re
import numpy as np

In [2]:
with open('forest.pickle','rb') as f:
    forest1 = pickle.load(f)

In [3]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [4]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [5]:
num_recommendations = 5
db = pd.read_csv('papers.csv')
db['text'] = db['title'] + ' ' + db['abstract']
title = 'Using a neural net to instantiate a deformable model'
result = predict(title, db, 128 , num_recommendations, forest1)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.00599980354309082 seconds to query forest.

 Top Recommendation(s) is(are) 
 995     Neural Network Weight Matrix Synthesis Using O...
5       Using a neural net to instantiate a deformable...
5191    A Self-Organizing Integrated Segmentation and ...
2069    Analytic Solutions to the Formation of Feature...
2457    Inferring Neural Firing Rates from Spike Train...
Name: title, dtype: object
